# Example data anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,str,str,f64,str,str,bool
"""23552806617768…","""../../../../da…","""12478847011144…",1.2927e19,"""53181871686849…","""73486916771591…",false
"""11203646027416…","""../../../../da…","""51331588507631…",1.4856e19,"""53181871686849…","""77524152222441…",true
"""16291894748580…","""../../../../da…","""17598768569952…",5.6458e17,"""49593458876464…","""73486916771591…",false
"""15020103529459…","""../../../../da…","""51331588507631…",4.0723e18,"""28126977189983…","""73486916771591…",false
"""14359294575251…","""../../../../da…","""12478847011144…",1.4677e19,"""28126977189983…","""88860952613008…",true
"""12338046023035…","""../../../../da…","""17598768569952…",1.6330e19,"""10909324345280…",null,false
"""13455182873306…","""../../../../da…","""17598768569952…",8.7675e18,"""28126977189983…","""88860952613008…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_City': 'PREDICTOR_0',
 'filename': 'filename',
 'Customer_MaritalStatus': 'PREDICTOR_2',
 'Customer_CLV': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,str,f64,str,str,str
"""16856143360333…","""../../../../da…","""50757166423645…",1.2927e19,"""FirstMortgage3…","""92632348364226…","""Rejected"""
"""16266417559788…","""../../../../da…","""14810279307003…",1.4856e19,"""FirstMortgage3…","""13628395378127…","""Accepted"""
"""88730172880564…","""../../../../da…","""10130082242291…",5.6458e17,"""MoneyMarketSav…","""92632348364226…","""Rejected"""
"""55980894655343…","""../../../../da…","""14810279307003…",4.0723e18,"""BasicChecking""","""92632348364226…","""Rejected"""
"""11419986483450…","""../../../../da…","""50757166423645…",1.4677e19,"""BasicChecking""","""17961677291928…","""Accepted"""
"""60812387458952…","""../../../../da…","""10130082242291…",1.6330e19,"""UPlusFinPerson…",null,"""Rejected"""
"""14076172515606…","""../../../../da…","""10130082242291…",8.7675e18,"""BasicChecking""","""17961677291928…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,filename,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,str,f64,str,str,str
"""85936090012872…","""../../../../da…","""33478057394244…",1.2927e19,"""FirstMortgage3…","""85536424774313…","""Rejected"""
"""17533367152692…","""../../../../da…","""12539211838357…",1.4856e19,"""FirstMortgage3…","""15192952310324…","""Accepted"""
"""47711058209114…","""../../../../da…","""91202705446091…",5.6458e17,"""MoneyMarketSav…","""85536424774313…","""Rejected"""
"""16679681786618…","""../../../../da…","""12539211838357…",4.0723e18,"""BasicChecking""","""85536424774313…","""Rejected"""
"""23145496808862…","""../../../../da…","""33478057394244…",1.4677e19,"""BasicChecking""","""37372273210997…","""Accepted"""
"""13915605785454…","""../../../../da…","""91202705446091…",1.6330e19,"""UPlusFinPerson…",null,"""Rejected"""
"""12374486961166…","""../../../../da…","""91202705446091…",8.7675e18,"""BasicChecking""","""37372273210997…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_City=PREDICTOR_0
filename=filename
Customer_MaritalStatus=PREDICTOR_2
Customer_CLV=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str
"""12991609008711…","""14208108123847…",1.2927e19,"""FirstMortgage3…","""14236990174176…","""Rejected"""
"""82372439126535…","""10454036768718…",1.4856e19,"""FirstMortgage3…","""17943252408857…","""Accepted"""
"""40157399825647…","""91565000238083…",5.6458e17,"""MoneyMarketSav…","""14236990174176…","""Rejected"""
"""16678734128694…","""10454036768718…",4.0723e18,"""BasicChecking""","""14236990174176…","""Rejected"""
"""15118223040954…","""14208108123847…",1.4677e19,"""BasicChecking""","""76625320922325…","""Accepted"""
"""16830035482199…","""91565000238083…",1.6330e19,"""UPlusFinPerson…",null,"""Rejected"""
"""11111467355378…","""91565000238083…",8.7675e18,"""BasicChecking""","""76625320922325…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

ComputeError: AttributeError: 'int' object has no attribute 'encode'